# 3주차 Day3 — 출처 정확성 검증 데모

> **블록3 심화용** | 강사 전용 데모 노트북
>
> 블록1에서 `ask("질문")`으로 답변을 봤다면,
> 이번에는 **출처가 정말 맞는지** PDF를 열어서 대조합니다.
>
> **강의 전 준비**: 아래 `[준비]` 셀 2개를 미리 실행해두세요.

---
## [준비] 시스템 초기화
> 강의 전에 미리 실행. 수강생 화면에는 보여주지 않음.

In [ ]:
# ============================================================
# [준비 1/2] 패키지 설치 + 환경설정
# 강의 전에 미리 실행. 수강생 화면에는 보여주지 않음.
# ============================================================

import os

# TF + protobuf 충돌 방지 (반드시 다른 import보다 먼저!)
os.environ["USE_TF"] = "0"
os.environ["USE_TORCH"] = "1"
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["HF_HUB_DISABLE_SYMLINKS_WARNING"] = "1"
# HuggingFace CDN 차단 우회
os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

!pip install -q langchain langchain-community langchain-huggingface langchain-text-splitters chromadb pypdf sentence-transformers

import os, textwrap
from IPython.display import display, HTML, Markdown

# === Google Gemini API 키 설정 ===
os.environ["GOOGLE_API_KEY"] = "AIzaSy...여기에_새_API_키_입력"

try:
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    pass

assert os.environ.get("GOOGLE_API_KEY"), "GOOGLE_API_KEY가 설정되지 않았습니다!"
print("환경설정 완료")

In [2]:
# ============================================================
# [준비 2/2] 벡터DB 구축 + RAG 체인 생성
# 강의 전에 미리 실행. 수강생 화면에는 보여주지 않음.
# ============================================================

import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import PromptTemplate
from google import genai as google_genai
from IPython.display import display, HTML

# --- 0. PDF 위치 자동 탐색 ---
pdf_name = "제품사양서_스마트냉장고_RF9000.pdf"
for root, dirs, files in os.walk(str(Path.cwd())):
    if pdf_name in files:
        os.chdir(root)
        break

# --- 1. PDF 로딩 ---
pdf_files = [
    "제품사양서_스마트냉장고_RF9000.pdf",
    "시험성적서_스마트냉장고_RF9000.pdf"
]
all_docs = []
for pdf_path in pdf_files:
    loader = PyPDFLoader(pdf_path)
    docs = loader.load()
    all_docs.extend(docs)
    print(f"{pdf_path.split('/')[-1]} → {len(docs)}페이지 로딩")
print(f"총 {len(all_docs)}페이지 로딩 완료")

# --- 2. 청킹 ---
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(all_docs)
print(f"{len(chunks)}개 청크로 분할")

# --- 3. 벡터DB (HuggingFace 임베딩) ---
print("임베딩 모델 로딩 중...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={"device": "cpu"}
)
vectorstore = Chroma.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
print("벡터DB 구축 완료")

# --- 4. LLM (Google Gemini) ---
gemini_client = google_genai.Client(api_key=os.environ["GOOGLE_API_KEY"])
GEMINI_MODEL = None
for candidate in ["gemini-2.0-flash-lite", "gemini-2.0-flash-001", "gemini-2.5-flash"]:
    try:
        gemini_client.models.generate_content(model=candidate, contents="test")
        GEMINI_MODEL = candidate
        print(f"LLM 모델: {candidate}")
        break
    except Exception:
        pass

# --- 5. 프롬프트 ---
basic_prompt = PromptTemplate(
    template="다음 문서를 참고하여 질문에 답하세요.\n\n문서 내용: {context}\n\n질문: {query}\n\n답변:",
    input_variables=["context", "query"]
)

custom_prompt = PromptTemplate(
    template="""당신은 제조업 기술문서 전문 AI 어시스턴트입니다.
다음 문서 내용만을 참고하여 질문에 정확하게 답하세요.

문서 내용:
{context}

질문: {query}

답변 규칙:
1. 반드시 문서에 있는 정보만 사용하세요
2. 문서에 없는 내용이면 "해당 정보를 제공된 문서에서 찾을 수 없습니다"라고 답하세요
3. 수치를 인용할 때는 단위를 반드시 포함하세요
4. 가능하면 출처 페이지를 언급하세요

답변:""",
    input_variables=["context", "query"]
)

# --- 6. 출력 함수 ---
def ask(query):
    docs = retriever.invoke(query)
    context = "\n\n".join(doc.page_content for doc in docs)
    prompt_text = custom_prompt.format(context=context, query=query)
    answer = gemini_client.models.generate_content(model=GEMINI_MODEL, contents=prompt_text).text

    sources = []
    seen = set()
    for doc in docs:
        src = doc.metadata.get("source", "알 수 없음").split("/")[-1]
        page = doc.metadata.get("page", -1) + 1
        key = f"{src}_p{page}"
        if key not in seen:
            seen.add(key)
            snippet = doc.page_content[:80].replace("\n", " ").strip()
            sources.append((src, page, snippet))

    src_html = ""
    for i, (src, page, snippet) in enumerate(sources, 1):
        src_html += f"""
        <div style='margin:4px 0; padding:6px 12px; background:#f0e6ff; border-radius:6px; font-size:13px;'>
        <b>[{i}]</b> {src} — <b>p.{page}</b>
        <br><span style='color:#8888aa; font-size:11px;'>"...{snippet}..."</span>
        </div>"""

    html = f"""
    <div style='max-width:800px; font-family:Arial,sans-serif;'>
    <div style='background:#4A1A6B; color:white; padding:12px 20px; border-radius:10px 10px 0 0; font-size:14px;'>
    <b>Q:</b> {query}
    </div>
    <div style='background:white; border:2px solid #E8DEF8; padding:16px 20px;'>
    <div style='font-size:15px; line-height:1.6; color:#1A1A2E;'>
    <b>답변:</b><br>{answer}
    </div>
    </div>
    <div style='background:#f5f1fa; padding:12px 20px; border-radius:0 0 10px 10px; border:1px solid #E8DEF8; border-top:none;'>
    <div style='font-size:13px; color:#7B2FBE; font-weight:bold; margin-bottom:6px;'>참조 출처:</div>
    {src_html}
    </div>
    </div>
    """
    display(HTML(html))


def compare(query):
    docs = retriever.invoke(query)
    context = "\n\n".join(doc.page_content for doc in docs)

    r_basic = gemini_client.models.generate_content(
        model=GEMINI_MODEL,
        contents=basic_prompt.format(context=context, query=query)
    ).text
    r_custom = gemini_client.models.generate_content(
        model=GEMINI_MODEL,
        contents=custom_prompt.format(context=context, query=query)
    ).text

    html = f"""
    <div style='max-width:800px; font-family:Arial,sans-serif;'>
    <div style='background:#4A1A6B; color:white; padding:12px 20px; border-radius:10px 10px 0 0; font-size:14px;'>
    <b>Q:</b> {query}
    </div>
    <div style='display:flex; gap:0;'>
    <div style='flex:1; background:#fff; border:2px solid #ccc; padding:14px; border-radius:0 0 0 10px;'>
    <div style='background:#8888aa; color:white; padding:4px 10px; border-radius:4px; font-size:12px; font-weight:bold; margin-bottom:8px; text-align:center;'>기본 체인</div>
    <div style='font-size:14px; line-height:1.5; color:#4A4A6A;'>{r_basic}</div>
    </div>
    <div style='flex:1; background:#fff; border:2px solid #7B2FBE; padding:14px; border-radius:0 0 10px 0;'>
    <div style='background:#7B2FBE; color:white; padding:4px 10px; border-radius:4px; font-size:12px; font-weight:bold; margin-bottom:8px; text-align:center;'>커스텀 체인</div>
    <div style='font-size:14px; line-height:1.5; color:#1A1A2E; font-weight:500;'>{r_custom}</div>
    </div>
    </div>
    </div>
    """
    display(HTML(html))


print("")
print("시스템 준비 완료")
print('ask("질문") → 답변 + 출처')
print('compare("질문") → 기본 vs 커스텀 비교')

제품사양서_스마트냉장고_RF9000.pdf → 4페이지 로딩
시험성적서_스마트냉장고_RF9000.pdf → 3페이지 로딩
총 7페이지 로딩 완료
11개 청크로 분할
임베딩 모델 로딩 중...
벡터DB 구축 완료
LLM 모델: gemini-2.5-flash

시스템 준비 완료
ask("질문") → 답변 + 출처
compare("질문") → 기본 vs 커스텀 비교


---
---
# 출처 검증 데모 시작

> **여기부터 수강생에게 보여줍니다.**
>
> 블록1에서 `ask("질문")`의 답변을 봤습니다.
> 이번에는 **출처가 정말 맞는지** 같이 확인합니다.
>
> **수강생에게**: 사양서 PDF와 시험성적서 PDF를 열어두세요.

## 데모 ① 사양서 질문 → PDF에서 출처 대조

In [3]:
ask("RF9000의 소비전력은 얼마인가요?")

> **사양서 p.2**를 열어보세요. 소비전력 36W가 실제로 있나요?

In [ ]:
ask("E3 에러코드는 무엇인가요? 원인과 조치 방법을 알려주세요.")

> **사양서 p.4** 에러코드표에서 E3을 찾아보세요. 일치하나요?

In [ ]:
ask("냉장실과 냉동실의 용량 차이는 얼마인가요?")

> 계산이 필요한 질문입니다. **사양서 p.2**에서 냉장 524L, 냉동 344L → 차이 180L이 맞는지 검증해보세요.

## 데모 ② 시험성적서 질문 + 교차 참조

In [4]:
ask("에너지효율 등급과 월간 소비전력량은?")

> 출처가 **시험성적서**로 나왔나요? 사양서가 아닙니다. **시험성적서 p.2**에서 확인해보세요.

In [ ]:
ask("이 냉장고의 소비전력과 에너지효율 등급을 함께 알려주세요")

> **핵심!** 소비전력은 사양서, 에너지효율은 시험성적서 — 출처에 두 문서가 함께 나오면 **교차 참조**가 된 겁니다.

## 데모 ③ 문서에 없는 질문을 하면?

In [5]:
ask("RF9000의 판매 가격은 얼마인가요?")

> 가격은 사양서에도 시험성적서에도 없습니다.
> LLM만 쓰면 "약 200만원" 같은 답을 지어내지만, RAG는 "찾을 수 없다"고 합니다.
> 이것이 **할루시네이션 방지** — 내일 Day4에서 더 깊이 다룹니다.

In [ ]:
ask("RF9000과 RF8000의 성능 차이는?")

> RF8000은 문서에 존재하지 않는 제품입니다. 어떻게 답했나요?

## 데모 ④ 수강생 자유 질문

> **수강생에게 질문을 받아서 아래 셀에 타이핑하세요.**
>
> 질문 후 함께 PDF를 열어서 출처가 맞는지 대조합니다.

In [ ]:
ask("")

In [ ]:
ask("")

In [ ]:
ask("")

---
## 강사 전용: 추가 질의 모음

> 시간이 남거나 수강생 질문에 대응할 때 사용

In [6]:
# 교차 참조 질문
ask("IEC 60335 기준으로 절연저항은 적합한가요?")

In [ ]:
# 긴 답변 — 에러코드 전체 목록
ask("E1부터 E9까지 모든 에러코드의 명칭을 알려주세요.")

In [ ]:
# 기능 관련
ask("FoodAI 식품인식 시스템의 인식 정확도는?")

In [ ]:
# 소음 수준 — 시험성적서
ask("냉장 운전 소음 측정값은?")

In [ ]:
# 유지보수 관련
ask("탈취 필터 교체 주기는 얼마인가요?")

In [ ]:
# 복합 교차 질문
ask("이 냉장고의 크기, 무게, 그리고 냉각 성능 시험 결과를 알려주세요")

In [ ]:
# EMC 관련
ask("전도성 방출 시험 결과가 기준 이내인가요?")